[Notebook of task](https://github.com/DistributedSystemsGroup/Algorithmic-Machine-Learning/blob/master/Challenges/Plankton/plankton_challenge.ipynb)


# Classification of Plankton based on features 


In [ ]:
!pip3 install --user 'mlens'

In [2]:
%matplotlib inline
# For configuration and jupiter
import os
import sys
import matplotlib
import implicit
# For data manipulation
import pandas as pd
import numpy as np
# For prediction
from tqdm import tqdm

basepath = "/mnt/datasets/plankton/flowcam/"

In [3]:
nativeDF = pd.read_csv(basepath + 'features_native.csv.gz')
labelsDF = pd.read_csv(basepath + 'meta.csv')

In [4]:
#print(nativeDF.isnull().sum().sort_values(ascending=False)[:10])
#print(skimagDF.isnull().sum().sort_values(ascending=False)[:6])

native_nan_cols = ['perimareaexc', 'feretareaexc', 'cdexc', 'skeleton_area',
                   'nb1_area', 'symetrieh_area', 'symetriev_area', 'convarea_area',
                  'nb2_area', 'nb3_area', ]


In [5]:
for col_name in native_nan_cols:
    nativeDF[col_name] = nativeDF[col_name].fillna(0)

labelsDF['objid'] = labelsDF['objid'].astype(np.int64, errors='ignore')
labelsDF['level1'] = labelsDF['level1'].fillna('No_level1_name')
labelsDF['level2'] = labelsDF['level2'].fillna('No_level2_name')

print(nativeDF.isnull().sum().any())
print(labelsDF.isnull().sum().any())

False
False


In [6]:
y = labelsDF['level2'] 
X = nativeDF

In [7]:
from sklearn.model_selection import train_test_split

# Divide train data into train and validation sets

seed = 42
test_size = 0.20

X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size = test_size, random_state = seed)

In [8]:
# without PCA, RandomForestClassifer is best and KNeighbours and MLP is awful. With PCA, MLP and KNeighbours perform similar to DecisionTree and RandomForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


model = RandomForestClassifier(n_estimators=200, n_jobs=-1, criterion='entropy')
model.fit(X_train, y_train)
model_predictions = model.predict(X_test)
model_f1 = f1_score(y_test, model_predictions, average='macro', labels=np.unique(model_predictions))
print("Final F1 Score: ", model_f1)

Final F1 Score:  0.5349062238006255


In [11]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import f1_score


model_et = ExtraTreesClassifier(n_estimators=100)
model_et.fit(X_train, y_train)
model_predictions_et = model_et.predict(X_test)
model_et_f1 = f1_score(y_test, model_predictions_et, average='macro', labels=np.unique(model_predictions_et))
print("Final F1 Score: ", model_et_f1)

Final F1 Score:  0.492734231948026


In [ ]:
'''# stacking 
from mlens.ensemble import Subsemble
from sklearn.linear_model import LogisticRegression
from mlens.metrics import make_scorer
from sklearn import preprocessing
from keras.utils import to_categorical

le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

rf = RandomForestClassifier(n_estimators=200, n_jobs=-1, criterion='entropy')
#kn = KNeighborsClassifier(3)
dt = DecisionTreeClassifier()
ab = AdaBoostClassifier()
mlpc = MLPClassifier(alpha=1)

lr = LogisticRegression()

seed = 42


f1 = make_scorer(f1_score, greater_is_better=True, average='macro', labels=np.unique(y_test))

ensemble = Subsemble(random_state=seed, folds=5, verbose = 2)

# Build the first layer
ensemble.add([rf, ab, dt, mlpc])
# Attach the final meta estimator
ensemble.add_meta(lr)
ensemble.fit(X_train, y_train)
e_preds = ensemble.predict(X_test)

e_f1 = f1_score(y_test_le, e_preds, average='macro', labels=np.unique(e_preds))
print("Stacked F1 Score: ", e_f1)'''

In [ ]:
# Use PCA for good results with MLPClassifer and KNN

# PCA performs best with a normalized feature set 
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()  
scX_train = sc.fit_transform(X_train)  
scX_test = sc.transform(X_test)  

from sklearn.decomposition import PCA

# 65 attributes in NativeDF
# 30 elements: 99,0%, 10 elements 83,4%, 20 elements: 96,1%

pca = PCA(.999)  
pcaX_train = pca.fit_transform(scX_train)  
pcaX_test = pca.transform(scX_test)  